# 1. Data Cleaning Process Household Pulse Survey

In [1]:
import pandas as pd
import constants
import os
import numpy as np

In [2]:
base_name='pulse2020_puf_'
df_lists=[]

for f in os.listdir('../../data/household/raw'):
    if base_name in f:
        data=pd.read_csv(os.path.join('../../data/household/raw',f), index_col=False)
        df_lists.append(data)

In [3]:
df_lists[0].head()

,SCRAM,WEEK,EST_ST,EST_MSA,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,AGENDER,RHISPANIC,...,COMP1,COMP2,COMP3,INTRNTAVAIL,INTRNT1,INTRNT2,INTRNT3,TSCHLHRS,TTCH_HRS,INCOME
0,V030000001S52011391390122,5,1,NaN,1239.935394,1973,2,1,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,3
1,V030000002S02020543300112,5,2,NaN,196.842234,1973,2,2,2,1,...,-99,1,-99,1,-99,1,-99,0.0,0.0,8
2,V030000002S02020880630122,5,2,NaN,295.425365,1951,2,1,2,2,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,-88
3,V030000002S02020999610122,5,2,NaN,1088.296594,1983,2,1,2,1,...,-99,1,-99,1,-99,1,-99,0.0,0.0,6
4,V030000005S58050092940112,5,5,NaN,20476.738688,1960,2,1,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,2


### 1. Studying the dataset

Public Use Data File (PUF) includes a replicate weight data file, and a data dictionary for every new release of the survey. The shape of the datasets has changed over time, depending on the number of surveyed people and the addition of new variables. 

#### Features
- 17 weeks, from April 23 to October 26
- Surveyed people between 50k-100k per survey
- Variables between 82 to 188
- Demographic variables
- Index variables: SCRAM (ID) and WEEK
- Spending variables
- Food variables
- Shopping variables
- Telework
- Trips
- Health
- Work variables
- Missing data designed as -88 and -99
- Mostly categorical data
- Require use of data dictionary to interpretate the name of columns and categories

Some interesting variables related to the spending of the stimulus payment can be found during weeks 7 to 12 from Phase 1 of the Survey and changes in shopping patterns are part of the new questions incorporated in Phase 2, from week 13 onwards. Although spending and shopping variables are not asked simultaneously to the population, they offer valious insights for our study as the survey tries to be representative along demographic variables. and they are pre-processed separately.

In [4]:
for df in df_lists:
    print(df.shape)

(105066, 82)
(91605, 105)
(90767, 105)
(101215, 82)
(86792, 105)
(83302, 84)
(73472, 105)
(109051, 188)
(88716, 188)
(132961, 82)
(41996, 82)
(95604, 188)
(110019, 188)
(99302, 188)
(74413, 82)
(98663, 105)
(108062, 105)


In [5]:
df_lists[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105066 entries, 0 to 105065
Data columns (total 82 columns):
SCRAM            105066 non-null object
WEEK             105066 non-null int64
EST_ST           105066 non-null int64
EST_MSA          31955 non-null float64
PWEIGHT          105066 non-null float64
TBIRTH_YEAR      105066 non-null int64
ABIRTH_YEAR      105066 non-null int64
EGENDER          105066 non-null int64
AGENDER          105066 non-null int64
RHISPANIC        105066 non-null int64
AHISPANIC        105066 non-null int64
RRACE            105066 non-null int64
ARACE            105066 non-null int64
EEDUC            105066 non-null int64
AEDUC            105066 non-null int64
MS               105066 non-null int64
THHLD_NUMPER     105066 non-null int64
AHHLD_NUMPER     105066 non-null int64
THHLD_NUMKID     105066 non-null int64
AHHLD_NUMKID     105066 non-null int64
THHLD_NUMADLT    105066 non-null int64
WRKLOSS          105066 non-null int64
EXPCTLOSS        105066 non

### 2. Relevant Variables

Food, Spending, Shopping

In [6]:
rel_var = constants.ID_VAR + constants.WEEK_VAR + constants.DEMOGRAPHICS_VARS + constants.EIP_VARS + constants.SHOPPING_VARS + constants.TRIPS_VAR

In [7]:
len(rel_var)

58

In [8]:
df_lists1 = []

for df in df_lists:
    df_copy = df.copy()
    for col in df_copy.columns:
        if col not in rel_var:
            df_copy.drop(columns=col, inplace=True)
    df_lists1.append(df_copy)

### Spending variables

1. EIP: Use of Economic Impact Payment (Stimulus)
    - Pay for expenses
    - Pay off debt
    - Add to savings
    - NA
2. EIPSPND: Spending use of Economic Impact Payment (Stimulus)
    - Food (groceries, eating out, take out)
    - Clothing, household supplies, household items, recreational, rent, mortgage, vehicle, saving or investments, charitable, credit card, loans, others.
    
For EDA, percent change of
- EIPSPND split by food and others (weekly and grouping by demographics)
- EIP split by food and others (weekly and grouping by demographics)

In [9]:
df_spending = pd.DataFrame()
for df in df_lists1:
    if 'EIP' in df.columns:
        df_spending = pd.concat([df_spending, df])

In [10]:
df_spending.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549361 entries, 0 to 108061
Data columns (total 28 columns):
SCRAM            549361 non-null object
WEEK             549361 non-null int64
EST_ST           549361 non-null int64
EST_MSA          167813 non-null float64
TBIRTH_YEAR      549361 non-null int64
EGENDER          549361 non-null int64
RHISPANIC        549361 non-null int64
RRACE            549361 non-null int64
EEDUC            549361 non-null int64
MS               549361 non-null int64
THHLD_NUMPER     549361 non-null int64
THHLD_NUMKID     549361 non-null int64
THHLD_NUMADLT    549361 non-null int64
EIP              549361 non-null int64
EIPSPND1         549361 non-null int64
EIPSPND2         549361 non-null int64
EIPSPND3         549361 non-null int64
EIPSPND4         549361 non-null int64
EIPSPND5         549361 non-null int64
EIPSPND6         549361 non-null int64
EIPSPND7         549361 non-null int64
EIPSPND8         549361 non-null int64
EIPSPND9         549361 non-

In [11]:
df_spending.groupby(['WEEK', 'EIP']).count()

SCRAM  EST_ST  EST_MSA  TBIRTH_YEAR  EGENDER  RHISPANIC  RRACE  \
WEEK EIP                                                                   
7    -99    655     655      223          655      655        655    655   
      1   39041   39041    11836        39041    39041      39041  39041   
      2   10303   10303     2802        10303    10303      10303  10303   
      3   10757   10757     2819        10757    10757      10757  10757   
      4   12716   12716     5477        12716    12716      12716  12716   
8    -99    936     936      320          936      936        936    936   
      1   58659   58659    17628        58659    58659      58659  58659   
      2   15055   15055     4148        15055    15055      15055  15055   
      3   15000   15000     4037        15000    15000      15000  15000   
      4   18412   18412     7937        18412    18412      18412  18412   
9    -99    922     922      307          922      922        922    922   
      1   55079   55079    15936        55079    55079      55079  55079   
      2   13532   13532     3439        13532    13532      13532  13532   
      3   12752   12752     3261        12752    12752      12752  12752   
      4   16378   16378     6768        16378    16378      16378  16378   
10   -99    785     785      266          785      785        785    785   
      1   50656   50656    14516        50656    50656      50656  50656   
      2   12018   12018     3059        12018    12018      12018  12018   
      3   11587   11587     2993        11587    11587      11587  11587   
      4   15721   15721     6466        15721    15721      15721  15721   
11   -99    856     856      258          856      856        856    856   
      1   51782   51782    14827        51782    51782      51782  51782   
      2   11881   11881     3094        11881    11881      11881  11881   
      3   11045   11045     2879        11045    11045      11045  11045   
      4   16041   16041     6420        16041    16041      16041  16041   
12   -99    839     839      277          839      839        839    839   
      1   49237   49237    14254        49237    49237      49237  49237   
      2   11306   11306     2867        11306    11306      11306  11306   
      3   10170   10170     2582        10170    10170      10170  10170   
      4   15240   15240     6117        15240    15240      15240  15240   

          EEDUC     MS  THHLD_NUMPER   ...    EIPSPND5  EIPSPND6  EIPSPND7  \
WEEK EIP                               ...                                   
7    -99    655    655           655   ...         655       655       655   
      1   39041  39041         39041   ...       39041     39041     39041   
      2   10303  10303         10303   ...       10303     10303     10303   
      3   10757  10757         10757   ...       10757     10757     10757   
      4   12716  12716         12716   ...       12716     12716     12716   
8    -99    936    936           936   ...         936       936       936   
      1   58659  58659         58659   ...       58659     58659     58659   
      2   15055  15055         15055   ...       15055     15055     15055   
      3   15000  15000         15000   ...       15000     15000     15000   
      4   18412  18412         18412   ...       18412     18412     18412   
9    -99    922    922           922   ...         922       922       922   
      1   55079  55079         55079   ...       55079     55079     55079   
      2   13532  13532         13532   ...       13532     13532     13532   
      3   12752  12752         12752   ...       12752     12752     12752   
      4   16378  16378         16378   ...       16378     16378     16378   
10   -99    785    785           785   ...         785       785       785   
      1   50656  50656         50656   ...       50656     50656     50656   
      2   12018  12018         12018   ...       12018     12018     12018   
      3   11587  

In [13]:
df_spending.groupby(['EST_MSA', 'EIP']).count()

SCRAM  WEEK  EST_ST  TBIRTH_YEAR  EGENDER  RHISPANIC  RRACE  \
EST_MSA EIP                                                                
12060.0 -99    109   109     109          109      109        109    109   
         1    5513  5513    5513         5513     5513       5513   5513   
         2    1227  1227    1227         1227     1227       1227   1227   
         3    1124  1124    1124         1124     1124       1124   1124   
         4    1973  1973    1973         1973     1973       1973   1973   
14460.0 -99    115   115     115          115      115        115    115   
         1    6127  6127    6127         6127     6127       6127   6127   
         2    1510  1510    1510         1510     1510       1510   1510   
         3    1640  1640    1640         1640     1640       1640   1640   
         4    3181  3181    3181         3181     3181       3181   3181   
16980.0 -99    107   107     107          107      107        107    107   
         1    5603  5603    5603         5603     5603       5603   5603   
         2    1402  1402    1402         1402     1402       1402   1402   
         3    1300  1300    1300         1300     1300       1300   1300   
         4    2297  2297    2297         2297     2297       2297   2297   
19100.0 -99     75    75      75           75       75         75     75   
         1    6124  6124    6124         6124     6124       6124   6124   
         2    1394  1394    1394         1394     1394       1394   1394   
         3    1113  1113    1113         1113     1113       1113   1113   
         4    2062  2062    2062         2062     2062       2062   2062   
19820.0 -99     95    95      95           95       95         95     95   
         1    5001  5001    5001         5001     5001       5001   5001   
         2    1103  1103    1103         1103     1103       1103   1103   
         3    1193  1193    1193         1193     1193       1193   1193   
         4    1332  1332    1332         1332     1332       1332   1332   
26420.0 -99    101   101     101          101      101        101    101   
         1    5892  5892    5892         5892     5892       5892   5892   
         2    1037  1037    1037         1037     1037       1037   1037   
         3     842   842     842          842      842        842    842   
         4    2278  2278    2278         2278     2278       2278   2278   
...            ...   ...     ...          ...      ...        ...    ...   
37980.0 -99    140   140     140          140      140        140    140   
         1    7272  7272    7272         7272     7272       7272   7272   
         2    1796  1796    1796         1796     1796       1796   1796   
         3    1696  1696    1696         1696     1696       1696   1696   
         4    2666  2666    2666         2666     2666       2666   2666   
38060.0 -99     84    84      84           84       84         84     84   
         1    5792  5792    5792         5792     5792       5792   5792   
         2    1393  1393    1393         1393     1393       1393   1393   
         3    1388  1388    1388         1388     1388       1388   1388   
         4    1718  1718    1718         1718     1718       1718   1718   
40140.0 -99     82    82      82           82       82         82     82   
         1    5191  5191    5191         5191     5191       5191   5191   
         2    1059  1059    1059         1059     1059       1059   1059   
         3     796   796     796          796      796        796    796   
         4    1200  1200    1200         1200     1200       1200   1200   
41860.0 -99    102   102     102          102      102        102    102   
         1    4383  4383    4383         4383     4383       4383   4383   
         2     786   786     786          786      786        786    786   
         3    1070  1070    1070         1070     1070       1070   1070   
         4    3652  3652    3652         3652     3652       

In [14]:
new_york = df_spending[df_spending.EST_MSA == 35620.0]
new_york.groupby(['WEEK', 'EIP']).count()

SCRAM  EST_ST  EST_MSA  TBIRTH_YEAR  EGENDER  RHISPANIC  RRACE  \
WEEK EIP                                                                   
7    -99     19      19       19           19       19         19     19   
      1    1175    1175     1175         1175     1175       1175   1175   
      2     252     252      252          252      252        252    252   
      3     210     210      210          210      210        210    210   
      4     601     601      601          601      601        601    601   
8    -99     38      38       38           38       38         38     38   
      1    1523    1523     1523         1523     1523       1523   1523   
      2     321     321      321          321      321        321    321   
      3     271     271      271          271      271        271    271   
      4     736     736      736          736      736        736    736   
9    -99     36      36       36           36       36         36     36   
      1    1370    1370     1370         1370     1370       1370   1370   
      2     262     262      262          262      262        262    262   
      3     218     218      218          218      218        218    218   
      4     737     737      737          737      737        737    737   
10   -99     24      24       24           24       24         24     24   
      1    1342    1342     1342         1342     1342       1342   1342   
      2     248     248      248          248      248        248    248   
      3     217     217      217          217      217        217    217   
      4     726     726      726          726      726        726    726   
11   -99     26      26       26           26       26         26     26   
      1    1372    1372     1372         1372     1372       1372   1372   
      2     291     291      291          291      291        291    291   
      3     232     232      232          232      232        232    232   
      4     702     702      702          702      702        702    702   
12   -99     29      29       29           29       29         29     29   
      1    1243    1243     1243         1243     1243       1243   1243   
      2     242     242      242          242      242        242    242   
      3     193     193      193          193      193        193    193   
      4     631     631      631          631      631        631    631   

          EEDUC    MS  THHLD_NUMPER   ...    EIPSPND5  EIPSPND6  EIPSPND7  \
WEEK EIP                              ...                                   
7    -99     19    19            19   ...          19        19        19   
      1    1175  1175          1175   ...        1175      1175      1175   
      2     252   252           252   ...         252       252       252   
      3     210   210           210   ...         210       210       210   
      4     601   601           601   ...         601       601       601   
8    -99     38    38            38   ...          38        38        38   
      1    1523  1523          1523   ...        1523      1523      1523   
      2     321   321           321   ...         321       321       321   
      3     271   271           271   ...         271       271       271   
      4     736   736           736   ...         736       736       736   
9    -99     36    36            36   ...          36        36        36   
      1    1370  1370          1370   ...        1370      1370      1370   
      2     262   262           262   ...         262       262       262   
      3     218   218           218   ...         218       218       218   
      4     737   737           737   ...         737       737       737   
10   -99     24    24            24   ...          24        24        24   
      1    1342  1342          1342   ...        1342      1342      1342   
      2     248   248           248   ...         248       248       248   
      3     217   217           217  

In [ ]:
#df_spending.replace({-88: np.nan, -99: np.nan}, inplace=True)

In [ ]:
#df = df.mask(df == 'PASS', '0')

In [9]:
# spending = []
# for df in df_lists1:
#     if 'EIPSPEND1' in df.columns:
#         spending.append(df)

Spending categories are not mutually excluyent, but we are interested specifically on food.

In [15]:
df_spending.groupby(['WEEK', 'EIPSPND1']).count()

SCRAM  EST_ST  EST_MSA  TBIRTH_YEAR  EGENDER  RHISPANIC  RRACE  \
WEEK EIPSPND1                                                                   
7    -99       24837   24837     6738        24837    24837      24837  24837   
     -88       13371   13371     5700        13371    13371      13371  13371   
      1        35264   35264    10719        35264    35264      35264  35264   
8    -99       35391   35391     9741        35391    35391      35391  35391   
     -88       19348   19348     8257        19348    19348      19348  19348   
      1        53323   53323    16072        53323    53323      53323  53323   
9    -99       31022   31022     8168        31022    31022      31022  31022   
     -88       17300   17300     7075        17300    17300      17300  17300   
      1        50341   50341    14468        50341    50341      50341  50341   
10   -99       27800   27800     7259        27800    27800      27800  27800   
     -88       16506   16506     6732        16506    16506      16506  16506   
      1        46461   46461    13309        46461    46461      46461  46461   
11   -99       27619   27619     7324        27619    27619      27619  27619   
     -88       16897   16897     6678        16897    16897      16897  16897   
      1        47089   47089    13476        47089    47089      47089  47089   
12   -99       25615   25615     6705        25615    25615      25615  25615   
     -88       16079   16079     6394        16079    16079      16079  16079   
      1        45098   45098    12998        45098    45098      45098  45098   

               EEDUC     MS  THHLD_NUMPER   ...    EIPSPND5  EIPSPND6  \
WEEK EIPSPND1                               ...                         
7    -99       24837  24837         24837   ...       24837     24837   
     -88       13371  13371         13371   ...       13371     13371   
      1        35264  35264         35264   ...       35264     35264   
8    -99       35391  35391         35391   ...       35391     35391   
     -88       19348  19348         19348   ...       19348     19348   
      1        53323  53323         53323   ...       53323     53323   
9    -99       31022  31022         31022   ...       31022     31022   
     -88       17300  17300         17300   ...       17300     17300   
      1        50341  50341         50341   ...       50341     50341   
10   -99       27800  27800         27800   ...       27800     27800   
     -88       16506  16506         16506   ...       16506     16506   
      1        46461  46461         46461   ...       46461     46461   
11   -99       27619  27619         27619   ...       27619     27619   
     -88       16897  16897         16897   ...       16897     16897   
      1        47089  47089         47089   ...       47089     47089   
12   -99       25615  25615         25615   ...       25615     25615   
     -88       16079  16079         16079   ...       16079     16079   
      1        45098  45098         45098   ...       45098     45098   

               EIPSPND7  EIPSPND8  EIPSPND9  EIPSPND10  EIPSPND11  EIPSPND12  \
WEEK EIPSPND1                                                                  
7    -99          24837     24837     24837      24837      24837      24837   
     -88          13371     13371     13371      13371      13371      13371   
      1           35264     35264     35264      35264      35264      35264   
8    -99          35391     35391     35391      35391      35391      35391   
     -88          19348     19348     19348      19348      19348      19348   
      1           53323     53323     53323      53323      53323      53323   
9    -99          31022     31022     31022      31022      31022      31022   
     -88          17300     17300     17300      17300      17300      17300   
      1           50341     50341     50341      50341      50341      50341   
10   -99          27800     27800     27800      2780

In [9]:
pd.concat([df_lists[0], df_lists[1]], sort=False)

,SCRAM,WEEK,EST_ST,EST_MSA,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,AGENDER,RHISPANIC,...,EIPSPND13,SPNDSRC1,SPNDSRC2,SPNDSRC3,SPNDSRC4,SPNDSRC5,SPNDSRC6,SPNDSRC7,CHILDFOOD,TSTDY_HRS
0,V030000001S52011391390122,5,1,NaN,1239.935394,1973,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V030000002S02020543300112,5,2,NaN,196.842234,1973,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V030000002S02020880630122,5,2,NaN,295.425365,1951,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V030000002S02020999610122,5,2,NaN,1088.296594,1983,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V030000005S58050092940112,5,5,NaN,20476.738688,1960,2,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,V030000005S91051510270112,5,5,NaN,1187.768788,1953,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,V030000008S19080964780122,5,8,NaN,2383.849099,1944,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,V030000008S53080818340112,5,41,NaN,4353.284629,1986,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,V030000009S30090182130112,5,12,NaN,1088.709353,1949,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,V030000009S70090246750112,5,9,NaN,2067.386361,1962,2,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_concat[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105066 entries, 0 to 105065
Data columns (total 13 columns):
SCRAM            105066 non-null object
EST_ST           105066 non-null int64
EST_MSA          31955 non-null float64
TBIRTH_YEAR      105066 non-null int64
EGENDER          105066 non-null int64
RHISPANIC        105066 non-null int64
RRACE            105066 non-null int64
EEDUC            105066 non-null int64
MS               105066 non-null int64
THHLD_NUMPER     105066 non-null int64
THHLD_NUMKID     105066 non-null int64
THHLD_NUMADLT    105066 non-null int64
INCOME           105066 non-null int64
dtypes: float64(1), int64(11), object(1)
memory usage: 10.4+ MB


In [ ]:
spending_data = pd.concat(df_concat)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [ ]:
final_data = pd.concat(df_lists)

In [ ]:
df = pd.read_csv(zf.open('pulse2020_puf_17.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_16.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_15.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_14.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_13.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_12.csv'))

In [8]:
type(df_list[0])

pandas.core.frame.DataFrame

In [ ]:
pulse_data = pd.concat(df_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [67]:
url_list[8]

'//www2.census.gov/programs-surveys/demo/datasets/hhp/2020/wk9/HPS_Week09_PUF_CSV.zip'

In [88]:
for i in range(len(url_list)):
    localDestination = "../data/household/"+url_list[i].split("Week")[1]
    print(localDestination[18:20]+'.csv')

17.csv
16.csv
15.csv
14.csv
13.csv
12.csv
11.csv
10.csv
09.csv
08.csv
07.csv
06.csv
05.csv
04.csv
03.csv
02.csv
01.csv


In [ ]:
print(localDestination[18:-4]+'.csv')

In [ ]:
final_data = pd.concat(df_lists, axis=0, sort=False)

In [59]:
weeks = [df_week1, df_week2, df_week3, df_week4, 
         df_week5, df_week6, df_week7, df_week8, 
         df_week9, df_week10, df_week11, df_week12, 
         df_week13]

pulse_df = pd.concat(weeks, axis=0, sort=False)
Preprocessing step 1: Replacement of weeks from integers to dat

'17'